In [6]:
!pip install spacy-universal-sentence-encoder
!pip install https://github.com/MartinoMensio/spacy-universal-sentence-encoder/releases/download/v0.4.1/en_use_md-0.4.1.tar.gz#en_use_md-0.4.1

     |████████████████████████████████| 173.9MB 194kB/s eta 0:00:011     |███████████████████████▊        | 128.9MB 8.7MB/s eta 0:00:06
     |████████████████████████████████| 112kB 6.9MB/s eta 0:00:01
     |████████████████████████████████| 3.3MB 13.3MB/s eta 0:00:01
     |████████████████████████████████| 133kB 8.4MB/s eta 0:00:01
     |████████████████████████████████| 71kB 7.4MB/s eta 0:00:011
     |████████████████████████████████| 15.6MB 16.2MB/s eta 0:00:01
     |████████████████████████████████| 1.0MB 6.4MB/s eta 0:00:01
     |████████████████████████████████| 10.6MB 21.0MB/s eta 0:00:01
     |████████████████████████████████| 471kB 6.3MB/s eta 0:00:01
     |████████████████████████████████| 61kB 5.6MB/s eta 0:00:01
     |████████████████████████████████| 3.0MB 4.5MB/s eta 0:00:01
     |████████████████████████████████| 51kB 8.8MB/s eta 0:00:01
     |████████████████████████████████| 143kB 8.1MB/s eta 0:00:01
     |████████████████████████████████| 102kB 7.9MB/s ta 0:00:011
   

In [63]:
from json_to_df import json_to_df
import pandas as pd
import numpy as np
import spacy

import os

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# 1 - Data Preprocessing

In [64]:
# df_paak = json_to_df('Lyrics_Anderson.Paak.json')
# df_abba = json_to_df('Lyrics_ABBA.json')
dfs = []
for file in os.listdir("./"):
    if file.endswith(".json"):
        fname = os.path.join('./', file)
        dfs.append(json_to_df(fname))

df = pd.concat(dfs)

In [65]:
df.sample(5)

,song title,lyrics,artist
4,Something,[Verse 1]\nSomething in the way she moves\nAtt...,The Beatles
26,Let’s Get Married,Sitting here\nWasting my time for you\nAnd I'm...,Al Green
17,Just a Little Bit,"I don't want much, I just want a little bit\nN...",Etta James
38,Bong Bong,"[Intro - RZA + U-God]]\nBong bong, bong bong, ...",RZA
14,Strawberry Fields Forever,"[Chorus]\nLet me take you down, 'cause I'm goi...",The Beatles


In [73]:
df.to_csv('all_songs.csv',index=False)

# 2 - USE and Model

In [66]:
nlp = spacy.load('en_use_md')

In [67]:
# create train_xs and train_ys
ys = df.artist

Xs = np.zeros((len(df), 512))
lyrics = df['lyrics']
for i, song in enumerate(lyrics):
    enc = nlp(song).vector
    Xs[i,:] = enc

print(Xs.shape, ys.shape)

(625, 512) (625,)


In [68]:
# split train and test
msk = np.random.rand(len(df)) < 0.8

train_xs = Xs[msk,:]
test_xs = Xs[~msk, :]

train_ys = ys[msk]
test_ys = ys[~msk]

print(train_xs.shape, test_ys.shape, train_ys.shape, test_ys.shape)

(509, 512) (116,) (509,) (116,)


In [71]:
model = LogisticRegression()
model.fit(train_xs, train_ys)

/Users/hiru/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/hiru/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [72]:
# in sample acc
train_y_pred = model.predict(train_xs)
print('training accuracy: {:.4}'.format(accuracy_score(train_ys, train_y_pred)))

test_y_pred = model.predict(test_xs)
print('testing accuracy: {:.4}'.format(accuracy_score(test_ys, test_y_pred)))

training accuracy: 0.7819
testing accuracy: 0.5
